## Alcholic Anonymous News articles extraction from Mumbai Mirror. Data Extraction of following parameters
- Headline
- Description
- Author
- Published_Date
- News
- URL
- Keywords
- Summary

### Importing the necessary Libraries

In [1]:
from selenium.webdriver.chrome.options import Options # enables options in web browser
from selenium import webdriver # web-based automation tool for Python
from newspaper import Article # Article scraping & curation
from bs4 import BeautifulSoup # Python library for pulling data out of HTML and XML files
from requests import get # standard for making HTTP requests in Python
import pandas as pd # library written for data manipulation and analysis
import sys, time #  System-specific parameters and functions

### Creating Empty lists for Alcholic Anonymous News Articles parameters data to be extracted

In [2]:
headlines, descriptions, dates, authors, news, keywords, summaries, urls = [], [], [], [], [], [], [], []

### Finding the total no.of.pages by total no.of articles from google search results¶

In [3]:
keyword = 'Alcoholics Anonymous site:mumbaimirror.indiatimes.com'

url = 'https://www.google.com/search?q=' + '+'.join(keyword.split())

options = Options()
options.headless = True
browser = webdriver.Chrome(options=options)
browser.get(url)

page = browser.page_source
soup = BeautifulSoup(page, 'lxml')
max_pages = round([int(s) for s in soup.select_one('#resultStats').text.split() if s.isdigit()][0]/10)

### Iterates max_pages value through while loop. Scraping the Articles urls

In [4]:
index = 0

while True:
    try:
        index +=1
        page = browser.page_source
        soup = BeautifulSoup(page, 'lxml')
        linky = [soup.select('.r')[i].a['href'] for i in range(len(soup.select('.r')))]
        urls.extend(linky)
        if index == max_pages:
            break
        browser.find_element_by_xpath('//*[@id="pnnext"]/span[2]').click()
        time.sleep(2)
        sys.stdout.write('\r' + str(index) + ' : ' + str(max_pages) + '\r')
        sys.stdout.flush()
    except:
        pass
    
browser.quit()

### To remove duplicates urls entries in the list by executing below line

In [5]:
urls = list(dict.fromkeys(urls))
print(len(urls), type(urls))

24 <class 'list'>


### Iterates urls through for loop. Scraping the Articles with above parameters

In [6]:
%%time
for index, url in enumerate(urls):
    try:
        # Parse the url to NewsPlease 
        soup = BeautifulSoup(get(url).text, 'lxml')
        article = Article(url)
        article.download()
        article.parse()
        article.nlp()
        
        # Extracts the Headlines
        try:
            try:
                headlines.append(article.title.strip())
            except:
                headlines.append(soup.select_one('meta[property="og:title"]')['content'].strip())
        except:
            headlines.append(None)
            
        # Extracts the Descriptions    
        try:
            try:
                descriptions.append(soup.select_one('meta[property="og:description"]')['content'].strip().replace('\n', ' '))
            except:
                descriptions.append(article.meta_description.strip())
        except:
            descriptions.append(None)
            
        # Extracts the Authors
        try:
            try:
                authors.append(soup.select_one('meta[name="author"]')['content'].strip())
            except:
                authors.append(article.authors.strip())
        except:
            authors.append(None)
        
        # Extracts the published dates
        try:
            try:
                dates.append(str(article.publish_date))
            except:
                dates.append(soup.select_one('meta[property="article:published_time"]')['content'].strip())
        except:
            dates.append(None)
            
        # Extracts the news articles
        try:
            try:
                news.append(soup.select_one('.storyText').text.replace('\n', '').strip())
            except:
                news.append(article.text.strip())
        except:
            news.append(None)
            
        # Extracts Keywords and Summaries    
        try:
            keywords.append(article.keywords)
            summaries.append(article.summary)
        except:
            keywords.append(None)
            summaries.append(None)
            
    except:
        headlines.append(None)
        descriptions.append(None)
        authors.append(None)
        dates.append(None)
        news.append(None)
        keywords.append(None)
        summaries.append(None)

    sys.stdout.write('\r' + str(index) + ' : ' + str(url) + '\r')
    sys.stdout.flush()

Wall time: 59.2 saimirror.indiatimes.com/mumbai/other/ats-top-boss-gets-threat-call/articleshow/15872943.cms-ended-up-having-alcohol-and-unprotected-sex,-which-I-don't-rememberNow-she-says-she-is-pregnantHow-will-I-know-if-it-is-my-child/sexpertanswer.cms?cid=44881


### Checking Array Length of each list to create DataFrame

In [7]:
print(len(headlines), len(descriptions), len(authors), len(dates), len(news), len(keywords), len(summaries), len(urls))

24 24 24 24 24 24 24 24


### Creating a csv file after checking array length and if to url error creates NaN values in the rows and dropping it

In [8]:
tbl = pd.DataFrame({'Headlines' : headlines,
                    'Descriptions' : descriptions,
                    'Authors' : authors,
                    'Published_Dates' : dates, 
                    'Articles' : news,
                    'Keywords' : keywords,
                    'Summaries' : summaries,})

tbl.to_csv('Mumbai Mirror.csv', index=False)
tbl.head()

Headlines  \
0  Alcoholic Anonymous asked to vacate civic prem...   
1  Heartbreak cure: Why you need to celebrate the...   
2                 Alcoholic cops, go for AA meetings   
3  Alcoholic deaddiction awareness programme held...   
4                      Joseph Pereira: Mumbai Heroes   

                                        Descriptions Authors Published_Dates  \
0  The organisation runs its national head office...    None            None   
1  Love is like a drug — romantic love causes cha...    None            None   
2  This refers to 'CP set to reassign 250 alcohol...    None            None   
3  Vashi:  A group of members of Alcoholic Anonym...    None            None   
4  For 36 years now, Fr Joseph Pereira has been u...    None            None   

                                            Articles  \
0  Alcoholics Anonymous\n\nHelen Keller\n\nspread...   
1  Love is like a drug — romantic love causes cha...   
2  This refers to 'CP set to reassign 250 alcohol...   
3  motivating talk: The AA members explained the ...   
4  A Father’s healing touch\n\nFather Joseph Pere...   

                                            Keywords  \
0  [21, school, classrooms, schools, civic, premi...   
1  [pain, know, past, faith, moment, feel, future...   
2  [refers, mm, bomb, cp, aa, force, alcoholic, n...   
3  [stn, quit, alcohol, deaddiction, vashi, polic...   
4  [father, fr, foundation, pereira, joe, rehabil...   

                                           Summaries  
0  I will give them apositive response,” he said....  
1  To know that, one day, even this difficult and...  
2  This refers to 'CP set to reassign 250 alcohol...  
3  The members of AA aimed at achieving their pri...  
4  A Father’s healing touchFather Joseph PereiraG...